In [1]:
### TODOs ###
# DONE: Implement Semantic Ranking
# TODOs: Commonsense Query and Concept Expansion: Topics, Concepts, Synonyms
# TODOs: Targeted Retrieval with NLI over ADUs, Premises, Claims; discard non-ADUs

# DONE: News Data
# DONE: Add Concepts
# DONE: Cosine Semantic Search
# DONE: Prior Pre-processing, tokenization and sentence segmentation to speed processing
# TODOs: Domain Restrict. Polarising social and political debate (Class labelling) only for higher-quality argument-knowledge set.
# TODOs: News, Political, Sociology and 'Good', 'Positive' counter-evidence Knowledge Base.
# TODOs: Bag of Topics Modelling
# TODOs: Implement as a Class

# TODOs: Keyphrase Selection
# DONE: Manage Duplicate Keywords
# DONE: Sentential Ranking
# DONE: Include Topic Label
# DONE: Include Concept Label
# DONE: Add News
# TODOs: Targeted Retreival with Semantic Graphs
# TODOs: Target Argumentative Content Only
# TODOs: Targeted Argument Content: Adus + Extractive Summary
# TODOs: Query Expansion
# TODOs: Multi-Field Search
# TODOs: Additional News and Knowledge Sources

In [2]:
### INIT LOGGING ###
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ARGUMENT-EXTRACTOR")

In [3]:
### INIT KNOWLEDGEBASE ###
from src.utils.elastic_db import ElasticDB

PORT = "http://localhost:9200"
db = ElasticDB(elastic_port=PORT)

INFO:src.utils.elastic_db:Connecting to http://localhost:9200 
INFO:src.utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [4]:
### NLP FUNCTIONS ###
from src.utils.utils import tokeniser, sentences_segment

In [5]:
### ADU CLASSIFIER ###
# import os
# path = "/Users/joshua.sheppard/PycharmProjects/countaBot/"
# os.chdir(path)

from src.detection.adu_classifier import predict

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


claim


In [6]:
### LOAD DATASETS ###
import json
import os

root = "/Users/joshua.sheppard/PycharmProjects/countaBot"
os.chdir(root)

args = [json.loads(ln) for ln in open("./src/data/processed/cmv_processed.jsonl")]
mined_args = [json.loads(ln) for ln in open("./src/data/processed/cmv_argument_extraction.jsonl")]
topics = [json.loads(ln) for ln in open("./src/data/processed/argument_topic_concept.jsonl")]
concepts = [json.loads(ln) for ln in open("./src/data/processed/argument_concept.jsonl")]

# args = [json.loads(ln) for ln in open("../data/processed/cmv_processed.jsonl")]
# mined_args = [json.loads(ln) for ln in open("../data/processed/cmv_argument_extraction.jsonl")]
# topics = [json.loads(ln) for ln in open("../data/processed/argument_topic_concept.jsonl")]
# concepts = [json.loads(ln) for ln in open("../data/processed/argument_concept.jsonl")]

In [7]:
print("ARGS:", len(args), " MINED-ARGS:",  len(mined_args), " TOPICS:", len(topics), " CONCEPTS: ", len(concepts))

ARGS: 10303  MINED-ARGS: 10303  TOPICS: 5990  CONCEPTS:  5990


In [7]:
### OPERATE ON A UNIQUE SET ###
import pandas as pd

# def unique_entries(args, key="id"):
#     data_ = pd.DataFrame(args)
#     unique = data_.drop_duplicates(subset=key)
#
#     unique_ = []
#     for _, i in unique.iterrows():
#         unique_.append({
#             "id": i["id"],
#             "claim": i["claim"],
#             "argument": i["argument"],
#             "tgt_counter": i["tgt_counter"],
#         })
#
#     return unique_
#
# unique_args = unique_entries(mined_args)
# unique_args

In [8]:
#len(unique_args)

In [8]:
### INSPECT SUBJECT ARG ###
import random
_ = random.randint(0, len(mined_args))

arg = " ".join(i["sentence"] for i in mined_args[_]["argument"])
claim = mined_args[_]["claim"]["sentence"]

#print(mined_args[_])
print(_, "\n")
print("CLAIM: ", claim, "\n")
print("ARG: ", arg, "\n")

9057 

CLAIM:  i believe that drag performers are participating in the equivalent of blackfaceminstrel shows for transgender individuals. . 

ARG:  a few years ago i went to a drag show at a local gay bar and an mtf friend of mine got in a huff about it. she explained that due to the exaggeration of masculinityfemininity that the shows portray as well as lampooning the idea of crossdressinggender bending it was damaging and offensive to trans individuals. she drew the comparison to blackface and how parallel exaggerations were used to demean people of african heritage at the turn of the th century. i countered with the notion that predominantly gay male performers were marginalized as well to which she pointed out that a nonwhite person in blackface would be equally offensive. i also tried to use the argument that they were also part of the transgender spectrum and she pointed out that a lot if not most of the performers were not experiencing any sort of gender dysphoria but were merel

In [9]:
### KEYPHRASE EXTRACTORS ###
from src.utils.keyphrase_extraction import yake_extract_keyphrase, summa_extract_keyphrase
import keybert

test = "Brazil's minimum income has increasingly been accepted."
ev_kp = yake_extract_keyphrase(test)
ev_kp_ = summa_extract_keyphrase(test)

test_2 = " "
ev_kp_2 = yake_extract_keyphrase(test_2)
ev_kp_2_ = summa_extract_keyphrase(test_2)

print(ev_kp)
print(ev_kp_)

# Assert can Handel Blanks
print(ev_kp_2)
print(ev_kp_2_)

INFO:summa.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
INFO:KEYPHRASE_EXTRACTOR:[Test Keyphrase: ] 
 ['heathrow airport', 'environmental impact', 'aviation']


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
['Brazil minimum income', 'minimum income has increasingly', 'Brazil minimum', 'increasingly been accepted', 'minimum income']
['minimum']
[]
[]


In [10]:
# def get_notion(notions_ids, notions_lst, arg_id, label):
#     notion_id = notions_ids.index(arg_id)
#     notion = notions_lst[notion_id][label]
#     return str(notion) if notion else None
#
# topic_ids = [json.loads(ln)["id"] for ln in open("./src/data/processed/argument_topic_concept.jsonl")]
#
# print(topic_ids.index("t3_30oi71"))
# print(topics[453])

453
{'id': 't3_30oi71', 'argument': 'Section I Why is Basic Income Increasingly Popular? Basic income is a policy that has broad support from both the progressive left and libertarian right. Centerleft economists including Paul Krugman have endorsed the scheme for various reasons. First BI is an effective antipoverty measure. BI also reduces inequality by redistributing income from capital to labor.', 'topic_label': 'incomes policy'}


In [15]:
from tqdm.notebook import tqdm
import multiprocessing
from src.detection.stance_classifier import sentence_stance, compare_stance
from src.detection.stance_classifier import sentence_stance
import time
import re

# Disable Huggingface Logging
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

topic_ids = [json.loads(ln)["id"] for ln in open("./src/data/processed/argument_topic_concept.jsonl")]
concept_ids = [json.loads(ln)["id"] for ln in open("./src/data/processed/argument_concept.jsonl")]

def clean(phrase):
    return re.sub(r"[,.;@#?!&$]+\ *", " ", phrase)

def get_notion(notions_ids, notions_lst, arg_id, label):
    notion_id = notions_ids.index(arg_id)
    notion = notions_lst[notion_id][label]
    return str(notion) if notion else None

### RETRIEVER ###
db = db
queries = []
retrieved_ev = []

# TODOs: Argumentative Sentence
# TODOs: Query Expansion
def search(mined, type="tgt_counter", l=10):
    id_ = mined["id"]
    claim = arg["claim"]

    topic = get_notion(topic_ids, topics, id_, "topic_label")
    concept = get_notion(concept_ids, concepts, id_, "concept_label")

    retrieved = []

    adu_count = 0
    targeted_response = []
    for adu in mined[type]:

        sentence = adu["sentence"]
        # if predict(sentence) != "premise":
        #     # Count ADUs for reference
        #     continue

        # TODOs: Check this isn't overriding continue
        adu_count += 1

        #kp = extract_keyphrase(sentence)
        kp = list(set(adu["kp"]))

        # TODOs: Common-sense Query Expansion
        query = []
        query.extend(kp)

        # Ensure topics and concepts are unpacked (extended) into query list, as lists, else string will unpack 'l', 'i', 'k', 'e', 't'
        query.extend([topic]) if topic else query
        query.extend([concept]) if concept else query
        query = list(set(query))

        # Note: Now query becomes a string - be careful
        query = ", ".join(i for i in query)
        # print(query)

        search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=l)]

        source = [i[0] for i in search]
        evidence = [i[1] for i in search]

        #print("query", query)
        merged = ", ".join(i for i in evidence)
        ev_kp = list(set(yake_extract_keyphrase(merged)))

        retrieved.append({"passages": evidence, "kp": [clean(i) for i in ev_kp], "source": source})

        targeted_response.append({"sentence": adu["sentence"], "selected_keyphrases": []})

    # TODOs: Implement yield without storing list
    return ({
        "id": id_,
        "claim": claim,
        "argument": mined["argument"],
        "tgt_counter": [i for i in targeted_response],
        "retrieved": [i for i in retrieved],
        "adu_count": adu_count
    })

# SINGLE ARGUMENT INSPECT
# SAMPLE = unique_args[0]
# results = search(SAMPLE)

# tic = time.time()
# SAMPLE = mined_args

# retrieved_ev = []
# with multiprocessing.Pool(8) as pool:
#     with tqdm(total=(len(SAMPLE)), position=0, leave=True) as pbar:
#         for arg in SAMPLE:
#             retrieved_ev.append(search(arg))
#             pbar.update()
#     toc = time.time()

INFO:STANCE_CLASSIFIER:[Initialised ... ]
INFO:STANCE_CLASSIFIER:[Test Stance ... ] 
 The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're in a relationship., PRO, better sex


In [29]:
retrieved_ev[4]

{'id': 't3_4gdj35',
 'claim': {'sentence': 'males and females are socially and emotionally incompatible and heterosexual relationships are unrealistic and farcical.',
  'kp': ['males and females']},
 'argument': [{'sentence': 'i am of the belief that there cannot exist a legitimate emotional bond between a male and a female that lead to a loving relationship that is one of romance.',
   'kp': ['legitimate emotional bond male',
    'loving relationship romance',
    'exist legitimate emotional',
    'female lead loving',
    'belief exist',
    'legitimate emotional bond'],
   'stance': 'PRO',
   'aspect': 'legitimate emotional bond male',
   'topic': 'couples',
   'concept': 'serious relations'},
  {'sentence': 'not once have i seen a couple that did not have an enormous gap in interests composure speech patterns and emotional engagement.',
   'kp': ['composure speech patterns emotional',
    'gap interests composure speech',
    'seen couple did',
    'engagement',
    'did enormous g

In [23]:
import copy
retrieved_ev_ = copy.deepcopy(retrieved_ev)

In [27]:
duration = toc - tic
print("TIME", duration)
retrieved_ev[0]

TIME 15349.299301862717


{'id': 't3_30oi71',
 'claim': {'sentence': 'we should strengthen the traditional safety net rather than replace it with basic income',
  'kp': ['traditional safety net']},
 'argument': [{'sentence': 'section i why is basic income increasingly popular?',
   'kp': ['basic income increasingly popular',
    'section basic income',
    'basic',
    'increasingly',
    'section',
    'income increasingly popular'],
   'stance': 'PRO',
   'aspect': 'basic income increasingly popular',
   'topic': 'incomes policy',
   'concept': 'increase in accumulated other comprehensive income'},
  {'sentence': 'basic income is a policy that has broad support from both the progressive left and libertarian right.',
   'kp': ['basic income',
    'basic income policy',
    'basic income policy broad',
    'income policy broad support',
    'support progressive left libertarian',
    'basic income'],
   'stance': 'PRO',
   'aspect': 'basic income',
   'topic': 'incomes policy',
   'concept': 'increase in accumu

In [25]:
len(retrieved_ev)

10303

In [14]:
# TODOs: Check Counter, Argument params pre-proccess
# TODOs: Process Argument pairs fully; Constrain at train time

_ = random.randint(0, len(mined_args))

# NOTE: ADU Opinion Classifier reduces returned argument response. This is ok.
print("Argument", len(retrieved_ev[_]["argument"]), "Retrieved", len(retrieved_ev[70]["retrieved"]))

# NOTE: ADU Opinion Classifier reduces returned argument response. This is ok.
print("Counter", len(retrieved_ev[_]["tgt_counter"]))

IndexError: list index out of range

In [26]:
# file_name = "cmv_retrieved"
# fout = open(f"./src/data/{file_name}.jsonl", "w")
#
# #with fout:
#     fout.write(json.dumps(retrieved_ev))
#
# logger.info(f"[{len(retrieved_ev)} Data Stored as {file_name}.jsonl]")

INFO:ARGUMENT-EXTRACTOR:[10303 Data Stored as cmv_retrieved.jsonl]


In [41]:
### REVIEW ###
# import os
# print(os.getcwd())

retrieved_ev_ = [json.loads(ln) for ln in open("./src/data/cmv_retrieved.jsonl", "r")][0]

_ = random.randint(0, len(review))
print(retrieved_ev_[_]["argument"], "\n")
print(retrieved_ev_[_]["retrieved"], "\n")
print(retrieved_ev_[_]["tgt_counter"], "\n")

[{'sentence': 'i understand with any post of this nature regarding some art or musical taste it might seem fruitless to attempt to rate something better than something else.', 'kp': ['art musical taste fruitless', 'taste fruitless attempt rate', 'rate better', 'taste', 'post nature regarding', 'art or musical'], 'stance': 'PRO', 'aspect': 'art musical taste fruitless', 'topic': None, 'concept': None}, {'sentence': 'however im going to attempt to first layout some critical definitions to narrow my argument then get into the actual argument itself.', 'kp': ['layout critical definitions', 'critical definitions narrow argument', 'actual argument', 'definitions', 'im going attempt', 'layout some critical'], 'stance': 'CON', 'aspect': 'layout critical definitions', 'topic': None, 'concept': None}, {'sentence': 'i hope this doesnt become too long and sorry if it does.', 'kp': ['hope doesnt long sorry', 'doesnt long sorry does', 'long', 'doesnt', 'sorry', 'hope this doesnt'], 'stance': 'CON', 

In [42]:
len(retrieved_ev_)

10303

In [43]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import torch
import time
import copy

# TODOs: Discard equivalent stance, per sentence
model = SentenceTransformer('all-MiniLM-L6-v2')
def cosine_similarity_(sentences):
    embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)

    cos = torch.nn.CosineSimilarity()
    scores = cos(embeddings[0], embeddings[1:])

    scored = []
    retrieved_sentences = sentences[1:]
    for sent, similarity in zip(retrieved_sentences, scores):
        scored.append((sent, similarity.numpy().item()))

    return scored

def rank_passages(ev, k=3):
    """ return ranked passages using cosine-similarity between the input-argument and the retrieved passages
        k determines the number of returned passages from the originally retrieved set.
    """
    #adus = [i["sentence"] for i in ev["argument"]]
    # Compare TGT with RETREIVED
    adus = [i["sentence"] for i in ev["tgt_counter"]]
    retrieved_passages = [i["passages"] for i in ev["retrieved"]]

    #print(retrieved_passages)

    # Merge
    # Output 1 x merged sentences object per ADU sentence, with k collected passages as a list of sentences
    merged_passages = []
    for passages in retrieved_passages:
        merged_sents = []
        # Iterate n x sentences for each k=5 retrieved passages
        for passage in passages:
            # Segment as a list of sentences
            sents = sentences_segment(passage)
            # Add sentences to merged_sentences object
            merged_sents.extend(sents)

        # Store merged sentence object for each ADU
        merged_passages.append(merged_sents)

    rank_retrieved = []
    # Rank n x merged sentences for each 1 x ADU
    for adu, merged in zip(adus, merged_passages):
        scored = []
        sentences = [adu]
        sentences.extend(merged)
        scored = cosine_similarity_(sentences)

        ranked_sents = sorted(scored, key=lambda x: x[1], reverse=True)

        # Select top-k sentences
        ranked_sents = ranked_sents[0:k]

        merged = ", ".join(i[0] for i in ranked_sents)
        merged_kp = yake_extract_keyphrase(merged)
        rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})

    #print("\n RANKED", rank_retrieved)
    return rank_retrieved

# TODOs: Join passages and sentence rank
### SCORE COSINE SIMILARITY ###
tic = time.time()
retrieved_ranked = copy.deepcopy(retrieved_ev_)
counta = 0
with tqdm(total=(len(retrieved_ev_)), position=0, leave=True) as pbar:
    for i in range(0, len(retrieved_ev_)):
        counta += 1
        retrieved_ranked[i]["retrieved"] = [i for i in rank_passages(retrieved_ev_[i])]
        pbar.update()

toc = time.time()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


  0%|          | 0/10303 [00:00<?, ?it/s]

In [44]:
retrieved_ranked_ = copy.deepcopy(retrieved_ranked)

In [39]:
subject = retrieved_ranked[2]

for i in subject:
    for _, j in zip(subject["tgt_counter"], subject["retrieved"]):
        print("COUNTER: ", _, "\n")
        print("EVIDENCE: ",j, "\n")

COUNTER:  {'sentence': 'first your disagreements about different funding ideas are not attacks on the basic income itself unless you were attacking resource taxes used to fund resource dividends a different but related idea to the basic income but you did not address that.', 'selected_keyphrases': []} 

EVIDENCE:  {'ranked_passages': 'Beginning in 2020, the fund will start paying out dividends to finance activities such as education, human resource development, economic development, infrastructure development, and tax relief., The PFD is a Basic Income in the form of a resource dividend., The resource is restricted in the sense that only earnings from the resource are used and not the principal.', 'kp': ['form of a resource', 'earnings from the resource', 'human resource development', 'fund will start paying', 'start paying out dividends']} 

COUNTER:  {'sentence': 'you want to keep the progressive income tax?', 'selected_keyphrases': []} 

EVIDENCE:  {'ranked_passages': 'Progressive i

In [107]:
duration = toc - tic
print(duration)
len(retrieved_ranked)

2585.4665830135345


5990

In [110]:
# NOTE: Zipping retrieved evidence, args
print(len(retrieved_ranked), len(retrieved_ev), len(args))

5990 5990 10303


In [45]:
_ = random.randint(0, len(retrieved_ev))
retrieved_ranked[_]

{'id': 't3_30oi71',
 'claim': {'sentence': 'we should strengthen the traditional safety net rather than replace it with basic income',
  'kp': ['traditional safety net']},
 'argument': [{'sentence': 'section i why is basic income increasingly popular?',
   'kp': ['basic income increasingly popular',
    'section basic income',
    'basic',
    'increasingly',
    'section',
    'income increasingly popular'],
   'stance': 'PRO',
   'aspect': 'basic income increasingly popular',
   'topic': 'incomes policy',
   'concept': 'increase in accumulated other comprehensive income'},
  {'sentence': 'basic income is a policy that has broad support from both the progressive left and libertarian right.',
   'kp': ['basic income',
    'basic income policy',
    'basic income policy broad',
    'income policy broad support',
    'support progressive left libertarian',
    'basic income'],
   'stance': 'PRO',
   'aspect': 'basic income',
   'topic': 'incomes policy',
   'concept': 'increase in accumu

In [119]:
print(_)

4001


In [47]:
file_name = "cmv_rr_"
fout = open(f"./src/data/processed/{file_name}.jsonl", "w")

# Deep_copies
rr = copy.deepcopy(retrieved_ranked)

with tqdm(total=(len(rr))) as pbar:
    with fout:
        for ln in rr:
            fout.write(json.dumps(ln))
            pbar.update()

logger.info(f"[{len(rr)} Data Stored as {file_name}.jsonl]")

  0%|          | 0/10303 [00:00<?, ?it/s]

INFO:ARGUMENT-EXTRACTOR:[10303 Data Stored as cmv_rr_.jsonl]


In [77]:
### INSPECT OUTPUT ###
rr_ = [json.loads(ln) for ln in open("./src/data/processed/cmv_rr.jsonl", "r")][0]

In [79]:
subject = rr_[random.randint(0, len(rr_))]

print("CLAIM: ", subject["claim"]["sentence"])
print("===========================================\n")
for i, j, k in zip(subject["argument"], subject["tgt_counter"], subject["retrieved"]):
    print("ARG: ", i["sentence"], "\n")
    print("COUNTER: ", j["sentence"], "\n")
    print("EVIDENCE: ", clean(k["ranked_passages"]).lower(), "\n")

CLAIM:  people who have sex with multiple partners im talking many many onenighters here are often deeply insecure or in some way damaged.

ARG:  i have a few friends who sleep around a lot. 

COUNTER:  while im sure this is true sometimes some people just have different values than you. 

EVIDENCE:  if such qualities are present in a person's world they tend to be apprehended they changed all of mine it's not just true for me it's true for most people following a 2017 constitutional court ruling on the status of same-sex marriage in taiwan chung stated 'we do not think people in china are seeking different values than people in taiwan  

ARG:  one person in particular told me yesterday that they had slept with people in the last two weeks im keeping this purposefully genderneutral to avoid a biased response. 

COUNTER:  that doesnt necessarily indicate any insecurity or damage.plenty of people just think sex is a fun activity that really has no moral or even any great emotional compon

In [80]:
# DONE: Keyphrase Selection
# TODOs: Full-run, arguments
import copy

### KEYPHRASE SELECTION OBJECT ###
_rr = copy.deepcopy(rr_)

In [76]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import torch

model = SentenceTransformer('all-MiniLM-L6-v2')

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [92]:
import re
def clean(phrase):
    return re.sub(r"[,.;@#?!&$]+\ *", " ", phrase)

def cosine_similarity_(sentences):
    embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)

    cos = torch.nn.CosineSimilarity()
    scores = cos(embeddings[0], embeddings[1:])

    scored = []
    retrieved_sentences = sentences[1:]
    for sent, similarity in zip(retrieved_sentences, scores):
        scored.append((sent, similarity.numpy().item()))

    return scored

def selected_keyphrases(arg):
    kps = [_["kp"] for _ in arg["retrieved"]]
    tgt_sentences = [_["sentence"] for _ in arg["tgt_counter"]]

    selected_kps = []
    for tgt, kp in zip(tgt_sentences, kps):
        vectors = [tgt]
        vectors.extend(kp)

        similarity = cosine_similarity_(vectors)
        #print("Before: ", [i[0] for i in similarity])
        selected = [i[0] for i in similarity if i[1] > 0.2]

        selected_kps.append(list(set(selected)))

    for _, j in zip(arg["tgt_counter"], selected_kps):
        _["selected_keyphrases"] = j

In [ ]:
### SELECTED KEYPHRASES ###
import tqdm as tqdm

SAMPLE = _rr
with tqdm.tqdm_notebook(total=(SAMPLE), position=0, leave=True) as pbar:
    for arg in SAMPLE:
        selected_keyphrases(arg)
        pbar.update()

/var/folders/gw/lkyxjfl92n9_j9h7mdgfz55c0000gq/T/ipykernel_8607/449248327.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm.tqdm_notebook(total=(SAMPLE), position=0, leave=True) as pbar:


In [94]:
_ = random.randint(0, 9)
_rr[_]["tgt_counter"]

[{'sentence': 'for what its worth there are two large biases here with you need to contend with.. the couples which you know and see will be influenced heavily with who you associate with and is not representative of relationships in general or the potential which some relationships can reach.',
  'selected_keyphrases': ['marriages report high',
   'marriages report high levels',
   'open marriages report high',
   'open marriages report']},
 {'sentence': 'this also depends heavily upon your judgment of those relationships.',
  'selected_keyphrases': ['depends on boys',
   'relationships.',
   'depends on relationships.',
   'depends']},
 {'sentence': 'which has the following problems you might not have complete indepth knowledge between the dynamics of these couples even if you did you might midjudge aspects of a relationship.',
  'selected_keyphrases': ['form an interdependent',
   'problems of infidelity',
   'form an interdependent relationship',
   'technology is interfering',
   

In [54]:
# WORK WITH DEEP COPIES

def overlap_kp(string, sub):
    count = start = 0
    while True:
        start = string.find(sub, start) + 1
        if start > 0:
            count+=1
        else:
            return count

## WORKING, YET REPLACED WITH SIMILARITY ###

# DONE: Similarity rank
# DONE: Add Stopwords
# stop = [i.strip() for i in open("./src/data/lexicon/stopwords.txt")]
# def selected_keyphrases(arg):
#     kps = [_["kp"] for _ in arg["retrieved"]]
#     tgt_sentences = [_["sentence"] for _ in arg["tgt_counter"]]
#
#     selected_kps = []
#
#     # Iterate per target sentence
#     for tgt, kp in zip(tgt_sentences, kps):
#         selected = []
#
#         for terms in kp:
#             singletons = terms.split()
#             for single in singletons:
#                 if single in stop:
#                     continue
#                 if overlap_kp(tgt.lower(), single) > 0:
#                     selected.append(terms)
#
#         selected_kps.append(list(set(selected)))
#
#     for _, j in zip(arg["tgt_counter"], selected_kps):
#         _["selected_keyphrases"] = j

In [201]:
#### TOY EXAMPLE: COMPATING SIMILARITY ###

counter = ["i dont see a comment on puberty yet so ill weigh in a bit.there is absolutely no reason to believe that all transwomen athletes have an advantage over their female counterparts as your assertion makes.the defining biological trigger that gives males an advantage over females is the presence of high amounts of testosterone"]

phrases = [
    'assertion makes the defining biological',
    'makes the defining biological trigger',
    'high amounts of testosterone',
    'assertion makes the defining',
    'makes the defining biological',
]

In [199]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: bert-base-nli-mean-tokens
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [202]:
embedding = counter
embedding.extend(phrases)

embedding


['i dont see a comment on puberty yet so ill weigh in a bit.there is absolutely no reason to believe that all transwomen athletes have an advantage over their female counterparts as your assertion makes.the defining biological trigger that gives males an advantage over females is the presence of high amounts of testosterone',
 'assertion makes the defining biological',
 'makes the defining biological trigger',
 'high amounts of testosterone',
 'assertion makes the defining',
 'makes the defining biological']

In [203]:
sentence_embeddings = model.encode(embedding)
result = cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[0.24159123, 0.24407683, 0.36673954, 0.12047476, 0.1817492 ]],
      dtype=float32)

In [195]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939


In [191]:
### EXTRACT LABELLED INSTANCES ###
# def extract_labelled(dict_):
#     res = []
#     count = 0
#     for i in dict_.items():
#         j, k = i
#
#         if k["selectec_keyphrases"] != []:
#             #res.append({"id": i[0], "argument": k["argument"], "label": k["label"]})
#             res.append(count)
#     return res
#
# counts = []
# for i in rr_train_:
#     sents = [_["sentence"] for _ in i["tgt_counter"]]
#     empties = [_ for _ in i["tgt_counter"] if _["selected_keyphrases"] != []]
#     counts.append((sents, empties))

# for i in rr_train_[0]["tgt_counter"]:
#     print(i["selected_keyphrases"])

for i in rr_train_["tgt_counter"]:
    print(i)

    break
    # print(i["selected_keyphrases"])

TypeError: list indices must be integers or slices, not str

In [193]:
for i in rr_train_:
    print(i["tgt_counter"])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [174]:
}import random
_ = random.randint(0, len(rr_train))
rr_train_[_]["tgt_counter"]

[{'sentence': 'i dont see a comment on puberty yet so ill weigh in a bit.there is absolutely no reason to believe that all transwomen athletes have an advantage over their female counterparts as your assertion makes.the defining biological trigger that gives males an advantage over females is the presence of high amounts of testosterone.',
  'kp': ['assertion makes.the defining biological',
   'makes.the defining biological trigger',
   'high amounts of testosterone',
   'assertion makes.the defining',
   'makes.the defining biological'],
  'stance': 'PRO',
  'aspect': 'assertion makes.the defining biological',
  'selected_keyphrases': []},
 {'sentence': 'testosterone changes the form of the body bone density and aids in the development and maintenance of muscle mass.however prepubescent children do not have high quantities of testosterone.',
  'kp': ['form of the body',
   'muscle mass.however prepubescent children',
   'body bone density',
   'bone density and aids',
   'maintenance 

In [140]:
print(_)

747


In [154]:
file_name = "cmv_rr_selected"
fout = open(f"./src/data/processed/{file_name}.jsonl", "w")

# Deep_copies
import copy
rr_selected = copy.deepcopy(rr_train_)

with fout:
    fout.write(json.dumps(rr_selected))
    fout.write("\n")

logger.info(f"[{len(rr_selected)} Data Stored as {file_name}.jsonl]")

INFO:ARGUMENT-EXTRACTOR:[5990 Data Stored as cmv_rr_selected.jsonl]


In [32]:
### PREVIOUS RETRIEVER ###

# from tqdm.notebook import tqdm
# import multiprocessing
# from src.detection.stance_classifier import sentence_stance, compare_stance
# from src.utils_.word_net_expansion import expand_query
# from src.detection.stance_classifier import sentence_stance
# # from multiprocessing.pool import ThreadPool as Pool
# import time
#
# # Disable Huggingface Logging
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
#
# ### RETRIEVER ###
# db = db
# queries = []
# retrieved_ev = []
#
# topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
# concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]
#
# # TODOs: Argumentative Sentence
# # TODOs: Query Expansion
# def search(mined):
#     id_ = mined["id"]
#     # print("\n", id_)
#
#     topic = arg["argument"][0]["topic"]
#     concept = arg["argument"][0]["concept"]
#
#     retrieve_len = 5
#     retrieved = []
#
#     # for adu in mined["argument"]:
#     for adu in mined["tgt_counter"]:
#
#         sentence = adu["sentence"]
#         if len(tokeniser(sentence)) <= 8:
#             continue
#
#         kp = list(set(adu["kp"]))
#         #print(kp)
#         # topic = adu["topic"]
#         # concept = adu["concept"]
#
#         kp.append(topic) if topic else kp
#         kp.append(concept) if concept else kp
#         # print(kp)
#
#         query = ", ".join(i for i in kp)
#
#         search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]
#
#         source = [i[0] for i in search]
#         evidence = [i[1] for i in search]
#
#         merged = ", ".join(i for i in evidence)
#         ev_kp = list(set(yake_extract_keyphrase(merged)))
#
#         retrieved.append({"passages": evidence, "kp": [i for i in ev_kp], "source": source})
#
#     # TODOs: Implement yield without storing list
#     return ({
#         "id": id_,
#         "argument": mined["argument"],
#         "tgt_counter": mined["tgt_counter"],
#         "retrieved": [i for i in retrieved],
#     })
#
# # SINGLE ARGUMENT INSPECT
# # SAMPLE = unique_args[0]
# # results = search(SAMPLE)
#
# tic = time.time()
# SAMPLE = unique_args[0:100]
#
# retrieved_ev = []
# with tqdm(total=(len(SAMPLE)), position=0, leave=True) as pbar:
#     for arg in SAMPLE:
#         retrieved_ev.append(search(arg))
#         pbar.update()
# toc = time.time()

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.072s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.084s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.070s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.069s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.045s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.105s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.038s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.051s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.025s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.037s]
INFO:elast

In [97]:
### QUERY EXPANSION ###
# TODOs: ConceptNet, pre-processing
# TODOs: WordNet, faster, also pre-processing
# from src.utils_.concept_net_expansion import ConceptNet
# sample = mined_args[5]
#
# concept_net = ConceptNet(api="http://api.conceptnet.io/", l=5)
# print(sample["claim"]["kp"])
# concept_ = sample["claim"]["kp"][0]

# terms = concept_.split()
# import os
# print(os.getcwd())
# stop = [i.strip() for i in open("./src/data/lexicon/stopwords.txt")]
# #print(stop)
#
# terms_ = [i for i in terms if i not in stop]
# expansion = [concept_net.get_similar(i) for i in terms_]
#
# expansion

['socially and emotionally incompatible', 'emotionally incompatible and heterosexual', 'incompatible and heterosexual relationships', 'heterosexual relationships are unrealistic', 'males and females']


In [267]:
# fout = open("../data/cmv_rr.jsonl", "w")
#
# args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
# sample = args[0:sample]
#
# # for i, j in zip(retrieved_ranked, sample):
# #     # Add counter to the dictionary (implicitly, i)
# #     i["counter"] = j["counter"]
# #     fout.write(json.dumps(i))
# #     fout.write("\n")

In [ ]:
# OLD
# kp = adu["kp"]
# topic = adu["topic"]
# concept = adu["concept"]
#
# kp.append(topic) if topic else kp
# kp.append(concept) if concept else kp
#
# query = ", ".join(i for i in adu["kp"])
# print(query)
#
# # TODOs: Add title field for all ES indices to enable multi-field search
# search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]
#
# #evidence = [i[1] for i in search]
# #source = [i[0] for i in search]
#
# evidence = [i[1] for i in search]
# ev_kp = yake_extract_keyphrase(evidence)
#
#         # try:
#         #     ev_kp = yake_extract_keyphrase(evidence)
#         # except:
#         #     ev_kp = [" "]
#
#         #retrieved.append({"passages": evidence, "kp": [i for i in ev_kp], "source": source})
#         #retrieved.append({"passages": evidence})
#
#
#
#     # TODOs: Implement yield without storing list
#     return ({
#         "id": id_,
#         "argument": mined["argument"],
#         "retrieved": [i for i in retrieved]
#     })
#
# for arg in unique_args[0:10]:
#     retrieved_ev.append(search(arg))
#
# # SAMPLE = unique_args[0:100]
# #
# # step = max(int(len(SAMPLE) / 10), 1)
# # BATCHES = [SAMPLE[i:i + step] for i in range(0, len(SAMPLE), step)]
# #
# # retrieved_ev = []
# # for idx, batch in enumerate(BATCHES):
# #     print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(batch)) + '-' * 25)
# #
# #     with multiprocessing.Pool(8) as pool:
# #         with tqdm(total=(len(batch))) as pbar:
# #             for arg in batch:
# #                 retrieved_ev.append(search(arg))
# #                 pbar.update()

In [44]:
# SINGLE SEARCH FUNCTION
####

# from tqdm.notebook import tqdm
# import multiprocessing
# from src.detection.stance_classifier import sentence_stance, compare_stance
# from src.utils_.word_net_expansion import expand_query
# from src.detection.stance_classifier import sentence_stance
# # from multiprocessing.pool import ThreadPool as Pool
# import time
#
# # Disable Huggingface Logging
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
#
# ### RETRIEVER ###
# db = db
# queries = []
# retrieved_ev = []
#
# topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
# concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]
#
# import random
# _ = random.randint(0, len(unique_args))
# sample = unique_args[_]
#
# # TODOs: Argumentative Sentence
# def search(mined):
#     id_ = mined["id"]
#     print("\n", id_)
#
#     retrieve_len = 5
#     retrieved = []
#     for adu in mined["argument"]:
#
#         sentence = adu["sentence"]
#         if len(tokeniser(sentence)) <= 8:
#             continue
#
#         kp = list(set(adu["kp"][0:5]))
#         topic = adu["topic"]
#         concept = adu["concept"]
#
#         kp.append(topic) if topic else kp
#         kp.append(concept) if concept else kp
#
#         query = ", ".join(i for i in adu["kp"])
#         print(query)
#         print(" ")
#         search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]
#
#         source = [i[0] for i in search]
#         evidence = [i[1] for i in search]
#
#         merged = ", ".join(i for i in evidence)
#         ev_kp = yake_extract_keyphrase(merged)
#         print(ev_kp)
#
#         retrieved.append({"passages": evidence, "kp": [i for i in ev_kp], "source": source})
#
#     # TODOs: Implement yield without storing list
#     return ({
#         "id": id_,
#         "argument": mined["argument"],
#         "retrieved": [i for i in retrieved]
#     })
#
# result = search(sample)

INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.054s]



 t3_5h86n0
election, coming, lot
 
['lot', 'lot of folks', 'election', 'folks', 'folks are living']
system of voting, prevent the tyranny, majority, dont, system
 


INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.149s]


['Electoral system', 'electoral system introduced', 'system', 'President of Chad', 'majority']
work with elections, structured to make, doesnt work, difficult to act, act against minority
 


INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.270s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.043s]


['British humanitarian aid', 'humanitarian aid worker', 'fellow aid workers', 'Sri Lanka', 'mass killings directed']
difficult to elect, make, difficult, elect
 
['sizeable African-American minority', 'African Americans began', 'system favored', 'favored the white', 'white majority']
authority to govern, system will fall, elected, collectively, authority
 


INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.125s]


['Saskatchewan Health Authority', 'Provincial Health Authority', 'Health Authority Act', 'Regional Health Authority', 'Health Authority']


In [18]:
# ### CHECK BLANKS ###
# args_ = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]
# ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]
#
# for j, k in zip(args_, ids):
#     if j == "":
#         print("blanks", j, k)

blanks  t3_3cm6jy
blanks  t3_1egv4k
blanks  t3_1egv4k
blanks  t3_5wjdve


In [30]:
# from src.detection.stance_classifier import sentence_stance, compare_stance
# from src.utils_.word_net_expansion import expand_query
# from src.detection.stance_classifier import sentence_stance
# import multiprocessing
# import json
# import time
#
# # TODOs: Adu, Counter + KP Extraction as 'Argument Mining' preprocessing module
#
# # Disable Huggingface Logging
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
#
# topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
# concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]
#
# def get_notion(notions_ids, notions_lst, arg_id, label):
#     notion_id = notions_ids.index(arg_id)
#     notion = notions_lst[notion_id][label]
#     return str(notion) if notion else None
#
# def extract_adus(arg_):
#     arg, id_ = arg_
#     print("\n", id_)
#
#     topic = get_notion(topic_ids, topics, id_, "topic_label")
#     concept = get_notion(concept_ids, concepts, id_, "concept_label")
#
#     adu_sents = sentences_segment(arg)
#
#     adus = []
#     for _ in adu_sents:
#         if len(tokeniser(_)) <= 8:
#             continue
#
#         try:
#             kp = extract_keyphrase(_)
#         except:
#             kp = [" "]
#
#         kp.append(topic) if topic else kp
#         kp.append(concept) if concept else kp
#         print(kp)
#
#         adu = {"sentence": _, "kp": [i for i in kp], "stance": sentence_stance(_, kp[0])}
#
#         adus.append(adu)
#
#     yield ({
#         "id": id_,
#         "argument": [i for i in adus]
#     })
#
# step = max(int(len(unique) / 10), 1)
# batches = [unique[i:i + step] for i in range(0, len(unique), step)]
#
# mined_args = []
# # TODOs: Remove Huggingface Warnings
#
# for idx, batch in enumerate(batches):
#     print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(batches)) + '-' * 25)
#
#     with multiprocessing.Pool(8) as pool:
#         with tqdm(total=(len(batch))) as pbar:
#             for arg in batch:
#                 mined_args.append([i for i in extract_adus(arg)])
#                 pbar.update()

-------------------------Batch 1/10-------------------------
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKEN

  0%|          | 0/599 [00:00<?, ?it/s]


 t3_5dv1q4
['topic', 'death', 'angle']
['opposed to Trump', 'work for Melania', 'opinion that places', 'places of public', 'public accommodation']
['people completely disgusting', 'religious fundamentalists denying', 'fundamentalists denying service', 'gay people completely', 'completely disgusting']
['problem', 'distinguish', 'cases']

 t3_5t56cr
['peace economics etc.', 'works journalism peace', 'journalism peace economics', 'liberal works journalism', 'wellknown liberal bias', 'australian democrats']
['Jim Simons mentioned', 'Simons mentioned indirectly', 'Democratic views represent', 'Jim Simons', 'Simons mentioned', 'australian democrats']
['potentially environmental failure', 'Republican Party', 'Party is objectively', 'current ideology', 'objectively untenable', 'australian democrats']
['progressive Democrats', 'corporate Democrats', 'comparison is relative', 'relative to progressive', 'Democrats not corporate', 'australian democrats']

 t3_3gys03
['retarded scientific progress

  0%|          | 0/599 [00:00<?, ?it/s]


 t3_3rd4r3
['generally liberal views', 'proBernie Sanders', 'rcirclejerk Redditors', 'liberal views', 'recurring theme']
['expecting free things', 'mission expecting balance', 'free things', 'goodsservicescontent creators', 'sort of mission']
['quote outofcontext sound', 'outofcontext sound bytes', 'slant left', 'level of social', 'social liberal']
['manchildren alike.So reddit', 'hive mind', 'mind is frowned', 'children and manchildren', 'manchildren alike.So']

 t3_6cn44s
['important traits', 'guy', 'rowdiness', 'important', 'traits', 'aggressiveness']
['public situation', 'rowdy guy', 'offensive if required', 'required either physically', 'physically or verbally', 'aggressiveness']
['physical if needed', 'guy can confront', 'people get physical', 'needed', 'teasing', 'aggressiveness']
['strong eye contact', 'strong eye', 'eye contact', 'strong', 'eye', 'aggressiveness']
['suck', 'hes', 'aggressiveness']

 t3_2tuej1
['Bill Nye', 'discussionargument on rdataisbeautiful', 'Bill', 'Nye

...


 t3_3wamx9
['codes formalized standards', 'private high schools', 'dress codes formalized', 'United States', 'buying clothes', 'excessive wear']
['preoccupied with clothes', 'free time', 'spend energy', 'energy or minutes', 'minutes picking', 'excessive wear']
['allowed to dress', 'invariably end', 'end up wearing', 'wearing clothes', 'students are allowed', 'excessive wear']
['foremost to foster', 'foster learning', 'mission', 'distracts from learning', 'learning is inherently', 'excessive wear']

 t3_667gvu
['considered mens issues', 'homelessness suicide custody', 'suicide custody jail', 'custody jail sentence', 'jail sentence length']
['feminist speaking', 'speaking', 'issues', 'addressed.This', 'feminist']
['occasional suicidal thought', 'strong distaste', 'socalled feminists', 'occasional suicidal', 'suicidal thought']
['naturally zerosum games', 'extension public support', 'zerosum games', 'objectivelyPublic attention', 'extension public']
['shelters breast cancer', 'breast can

  0%|          | 0/599 [00:00<?, ?it/s]


 t3_20ofb2
['friends found', 'back story', 'fight', 'back', 'story', 'fists']
['Punches were thrown', 'pretty badly', 'cheated on beat', 'beat friend', 'friend B pretty', 'fists']
['firing from jobs', 'girl who called', 'called friend', 'friend B admitted', 'admitted I dont', 'fists']
['fist fight settle', 'fist fight', 'fight settle', 'settle the tension', 'friends faster', 'fists']
['day life.Assuming consent', 'extreme harmokay guys', 'good points presented', 'mature debate', 'legal in hockey', 'fists']

 t3_22p8vc
['social constructs', 'disputing the existence', 'existence of social', 'constructs', 'clear', 'nervous origin']
['deeper significance', 'phenomenon of dismissing', 'dismissing a common', 'common belief', 'belief or behavior', 'nervous origin']
['natural factors', 'necessarily have arisen', 'today.My reasoning', 'play hundreds', 'hundreds of thousands', 'nervous origin']
['enduring social constructs', 'social constructs grew', 'instinctual behavior', 'language developed'

  0%|          | 0/599 [00:00<?, ?it/s]


 t3_5uutr2
['white people based', 'favor black people', 'word racist found', 'word discrimination found', 'ethnicity.So reddit change']

 t3_2rc3p3
['American patriots', 'mentality against immigration', 'general mentality', 'American', 'immigration']
['people shouldnt', 'shouldnt be allowed', 'line and americans', 'born', 'side']
['slogan Creating jobs', 'slogan Creating', 'Creating jobs', 'jobs for americans', 'ads and political']
['private corporations', 'understand why politics', 'americans to vote', 'prominent in ads', 'ads made']
['creating jobs', 'jobs for americans', 'jobs for people', 'americans is morally', 'morally superior']

 t3_1h1yj0
['sound more important', 'recognize utilize', 'puff word', 'important', 'recognize', 'unnecessary use']
['overcomplicating language', 'select few times', 'makes more sense', 'sense are negligible.What', 'utilize makes', 'unnecessary use']

 t3_2fdmwl
['tldr Hyperrealistic paintingsdrawings', 'Hyperrealistic paintingsdrawings offer', 'tldr Hy

In [28]:
# def get_topic(arg_id):
#     topic_id = topic_ids.index(arg_id)
#     topic = topics[topic_id]["topic_label"]
#     return str(topic) if topic else None
#
# def get_concept(arg_id):
#     concept_id = concept_ids.index(arg_id)
#     concept = concepts[concept_id]["concept_label"]
#     return str(concept) if concept else None

In [26]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

# TODOs: Fix Vectorizer Issue
# kb = KeyBERT()
# vectorizer = KeyphraseCountVectorizer()
# def extract_keyphrase(doc, n_gram=3, n_kp=3, use_mmr="False", use_maxsum="False"):
#     try:
#         kp = kb.extract_keywords(doc, keyphrase_ngram_range=(0, 3), stop_words="english", diversity=0.3,)
#         kp_ = kb.extract_keywords(doc, vectorizer=vectorizer, stop_words="english", diversity=0.3)
#
#     except:
#         return [" "]
#
#     # Concatonate, remove duplicates
#     kp = kp + kp_
#     kp = [i[0] for i in kp]
#     kp = list(set(kp))
#
#     return kp

In [ ]:
# # TODOs: Compute in Batches
# sample = unique[0:100]
# with multiprocessing.Pool(8) as pool:
#     with tqdm(total=(len(unique))) as pbar:
#         for arg in unique:
#             mined_args.append([i for i in extract_adus(arg)])
#             pbar.update()

In [ ]:
# def retrieved_evidence(mined, retrieve_len=5):
#     """ Retrieves Evidence from Knowledge base, returning a well-formed Retrieved Evidence Object
#     given an input Argument"""
#
#     id_ = mined["id"]
#     print("\n", id_)
#
#     retrieved = []
#     adus = []
#     for _ in mined["argument"]:
#         if len(tokeniser(_)) <= 8:
#             continue
#
#         kp = extract_keyphrase(_)
#         print(kp)
#         adu = {"sentence": _, "kp": [i for i in kp], "stance": sentence_stance(_, kp[0])}
#
#         kp.append(topic) if topic else kp
#         kp.append(concept) if concept else kp
#
#         query = ", ".join(i for i in kp)
#         print(query)
#
#         # TODOs: Add title field for all ES indices to enable multi-field search
#         search = [(i["_source"]["document"]["source"], i["_source"]["document"]["text"]) for i in db.search(query_=query, k=retrieve_len)]
#
#         evidence = [i[1] for i in search]
#         source = [i[0] for i in search]
#
#         ev_kp = extract_keyphrase(evidence)
#
#         retrieved.append({"passages": evidence, "kp": [i[0] for i in ev_kp], "source": source})
#         #retrieved.append({"passages": evidence, "source": source})
#         adus.append(adu)
#
#     return ({
#         "id": id_,
#         "argument": [i for i in adus],
#         "retrieved": [i for i in retrieved]
#     })

In [224]:
# from sentence_transformers import SentenceTransformer, util
# from sklearn.metrics.pairwise import cosine_similarity
# import torch
# import time
#
# model = SentenceTransformer('all-MiniLM-L6-v2')
# def cosine_similarity_(sentences):
#     embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)
#
#     cos = torch.nn.CosineSimilarity()
#     scores = cos(embeddings[0], embeddings[1:])
#
#     scored = []
#     retrieved_sentences = sentences[1:]
#     for sent, similarity in zip(retrieved_sentences, scores):
#         scored.append((sent, similarity.numpy().item()))
#
#     return scored
#
# def rank_passages(ev, k=3):
#     adus = [i["sentence"] for i in ev["argument"]]
#     retrieved_passages = [i["passages"] for i in ev["retrieved"]]
#
#     # Merge
#     # Output 1 x merged sentences object per ADU sentence, with k=5 collected passages as a list of sentences
#     merged_passages = []
#     for passages in retrieved_passages:
#         merged_sents = []
#         # Iterate n x sentences for each k=5 retrieved passages
#         for passage in passages:
#             # Segment as a list of sentences
#             sents = sentences_segment(passage)
#             # Add sentences to merged_sentences object
#             merged_sents.extend(sents)
#
#         # Store merged sentence object for each ADU
#         merged_passages.append(merged_sents)
#
#     rank_retrieved = []
#     # Rank n x merged sentences for each 1 x ADU
#     for adu, merged in zip(adus, merged_passages):
#         scored = []
#         sentences = [adu]
#         sentences.extend(merged)
#         scored = cosine_similarity_(sentences)
#
#         ranked_sents = sorted(scored, key=lambda x: x[1], reverse=True)
#
#         # Select top-k sentences
#         ranked_sents = ranked_sents[0:k]
#
#         merged = ", ".join(i[0] for i in ranked_sents)
#         merged_kp = extract_keyphrase(merged)
#         rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})
#
#     return rank_retrieved
#
# # TODO: Clean text
# # TODO: Collect unique Keyphrases per Argument
# rank_passages(retrieved_ev[0])
# #print(len(test["ranked_passages"][0]))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


[{'ranked_passages': "The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic income as a progressive social policy for the United Kingdom, and beyond'., Several British academics have been involved in the basic income debate., Her son Brandon Rhys Williams proposed a basic income to a parliamentary committee in 1982, and soon after that in 1984, the Basic Income Research Group, now the Citizen's Basic Income Trust, began to conduct and disseminate research on basic income.",
  'kp': ['organisation basic income uk',
   'basic income',
   'basic income debate',
   'basic income uk']},
 {'ranked_passages': "But far more of the contemporary support for basic income in the United States has come from the left of center, driving by people who see it as a major expansion of support for low-income people, as the late, Al Sheahan argued in his 2012 book, 'the Basic Income Guarantee': 'Your right' 'to economic security'., In the Czech Republic, unco

In [62]:
# Handle duplicates
# def rank_passages(ev, k=3):
#     """ Handles a Retrieved Evidence Object, returning the top-k passages for each ADU """
#     # Per Argument
#     # Index into Retrieved Evidence Object
#     adus = [i for i in ev[0]["argument"]]
#     retrieved = [i for i in ev[0]["retrieved"]]
#
#     #print(len(retrieved), len(adus))
#
#     # Rank k-returned passages for each ADU
#     r_retrieved = []
#     for adu, passage in zip(adus, retrieved):
#         scored = []
#         ranked_ev = []
#         for _, kp in zip(passage["evidence"], passage["kp"]):
#             scored.append((_, kp, cosine_similarity(str(adu), str(_))))
#
#         scored = sorted(scored, key=lambda x: x[2], reverse=True)[0:3]
#         for i, j, k in scored:
#             ranked_ev.append({"evidence": i, "kp": j, "similarity": k})
#
#         r_retrieved.append(ranked_ev)
#
#     return r_retrieved

# 1 Argument x 4 ADUs x 5 Retrieved Passages
# ranked = [i for i in rank_passages(retrieved_ev[3])]
# print(len(ranked))
# print(ranked)

4 4
4
[[{'evidence': 'Many technology experts and technology entrepreneurs have begun endorsing basic income in the 2000s and 2010s. These include Marshal Brain, Sam Altman, James Hughes, Facebook co-founder Chris Hughes, Elon Musk, and Mark Zuckerberg (in his 2017 Harvard commencement speech), and Jeremy Rifkin. The overriding theme among technologists who favor basic income is the belief that automation is creating an increasingly unstable labor market.', 'kp': ('technologists favor basic income', 0.6414), 'similarity': 0.3174712657928467}, {'evidence': "Committee member Lady Rhys-Williams argued that the incomes for adults should be more like a basic income. She was also the first to develop the negative income tax model. Her son Brandon Rhys Williams proposed a basic income to a parliamentary committee in 1982, and soon after that in 1984, the Basic Income Research Group, now the Citizen's Basic Income Trust, began to conduct and disseminate research on basic income.", 'kp': ('will

In [13]:
# from sentence_transformers import SentenceTransformer, util
# import torch
# import time
#
# model = SentenceTransformer('all-MiniLM-L6-v2')
#
# # TODOs: Join passages and sentence rank
# ### SCORE COSINE SIMILARITY ###
# def cosine_similarity(sent_1, sent_2):
#     sentences = [sent_1, sent_2]
#     embeddings = model.encode(sentences, convert_to_tensor=True, show_progress_bar=False)
#
#     cos = torch.nn.CosineSimilarity(dim=0)
#     score = cos(embeddings[0], embeddings[1])
#
#     return score.numpy().item()
#
# ### SCORE TF-KEYWORD OVERLAP ###
# def overlap_score(evidence_kp, adu_kp):
#     score = 0
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if ", ".join([i for i in adu_kp]).find(j) != -1: score += 1
#
#             else: continue
#     return score
#
# ### RANK PASSAGES ###
# def score_passages(ev):
#     for _ in range(0, len(ev["argument"])):
#         print(_)
#
# from collections import defaultdict
# def rank_passages(ev, k=2):
#     adus = [i for i in ev["argument"]]
#     retrieved = [i for i in ev["retrieved"]]
#
#     rank_retrieved = []
#     count = 0
#
#     for adu, passages in zip(adus, retrieved):
#         count += 1
#         scored = []
#
#         # 5 passages
#         for passage in passages["passages"]:
#             score = cosine_similarity(str(adu), str(passage))
#             scored.append((passage, score))
#
#         ranked_passages = sorted(scored, key=lambda x: x[1], reverse=True)
#         ranked_passages = ranked_passages[0:k]
#
#         merged = ", ".join(i[0] for i in ranked_passages)
#         merged_kp = extract_keyphrase(merged)
#         rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})
#
#     return rank_retrieved
#
# import copy
# ### UPDATE RETRIEVED OBJECT ###
# # for i in range(0, len(retrieved_ev)):
# #     retrieved_ranked[i]["retrieved"] = [i for i in rank_passages(retrieved_ev[i])]
#
# tic = time.time()
# retrieved_ranked = copy.deepcopy(retrieved_ev)
# with tqdm(total=(len(retrieved_ev)), position=0, leave=True) as pbar:
#     for i in range(0, len(retrieved_ev)):
#         retrieved_ranked[i]["retrieved"] = [i for i in rank_passages(retrieved_ev[i])]
#     pbar.update()
#
# toc = time.time()
# # duration = toc - tic
#
# retrieved_ranked

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


  0%|          | 0/10 [00:00<?, ?it/s]

[{'id': 't3_30oi71',
  'argument': [{'sentence': 'Section I Why is Basic Income Increasingly Popular?',
    'kp': ['basic income increasingly',
     'basic income',
     'section basic income'],
    'stance': 'NEUTRAL'},
   {'sentence': 'Basic income is a policy that has broad support from both the progressive left and libertarian right.',
    'kp': ['basic income', 'basic income policy', 'income policy'],
    'stance': 'PRO'},
   {'sentence': 'Centerleft economists including Paul Krugman have endorsed the scheme for various reasons.',
    'kp': ['krugman endorsed scheme',
     'centerleft economists including',
     'centerleft economists'],
    'stance': 'PRO'},
   {'sentence': 'BI also reduces inequality by redistributing income from capital to labor.',
    'kp': ['bi reduces inequality',
     'inequality redistributing income',
     'reduces inequality redistributing'],
    'stance': 'CON'}],
  'retrieved': [{'ranked_passages': "Several British academics have been involved in the b

In [202]:
# Fix KW extraction
# Fix Duplicates
# def rank_passages(ev, k=2):
#     adus = [i for i in ev["argument"]]
#     retrieved = [i for i in ev["retrieved"]]
#
#     rank_retrieved = []
#     count = 0
#
#     for adu, passages in zip(adus, retrieved):
#         count += 1
#         scored = []
#
#         # 5 passages
#         for passage in passages["passages"]:
#             score = cosine_similarity(str(adu), str(passage))
#             scored.append((passage, score))
#
#         ranked_passages = sorted(scored, key=lambda x: x[1], reverse=True)
#         ranked_passages = ranked_passages[0:k]
#
#         merged = ", ".join(i[0] for i in ranked_passages)
#         merged_kp = extract_keyphrase(merged)
#         rank_retrieved.append({"ranked_passages": merged, "kp": merged_kp})
#
#     return rank_retrieved
#
# ev = retrieved_ev[0]
# rank_passages(ev)

[{'ranked_passages': "Several British academics have been involved in the basic income debate. Among them the following:Organisations. The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic income as a progressive social policy for the United Kingdom, and beyond'., Basic Income. Delaney opposes implementing a basic income (also known as a universal basic income). Minimum wage.",
  'kp': ['organisation basic income uk',
   'basic income uk',
   'united kingdom basic income']},
 {'ranked_passages': "Several British academics have been involved in the basic income debate. Among them the following:Organisations. The organisation Basic Income UK is 'a collective of independent people promoting unconditional basic income as a progressive social policy for the United Kingdom, and beyond'., A basic income is defined in the report as a policy that guarantees all members of a society a minimum amount of income. One type of basic income considered is

In [458]:
# def fuck_you():
#     print("fuck you")
#
# fuck_you()
#
# def rank_(ev):
#     # Index into Retrieved Evidence Object
#     ev = ev[0]
#     adus = [i for i in ev[0]["argument"]]
#     retrieved = [i for i in ev[0]["retrieved"]]
#     k = 3
#     print("hello")
#     # # Rank k-returned passages for each ADU
#     # count = 0
#     # r_retrieved = []
#     # for adu, passage in zip(adus, retrieved):
#     #     count += 1
#     #     ranked_passages = []
#     #     for _ in passage["evidence"]:
#     #         print(_)
#     #         ranked_passages.append((_, cosine_similarity(adu, _)))
#     #         r_retrieved.append({"evidence": i, "similarity": k} for i, k in sorted(ranked_passages, key=lambda x: x[1], reverse=True)[0:k])
#     #         print(r_retrieved)
#
#     # return {
#     #     "r_retrieved": r_retrieved
#     # }
#
# #print(rank_(retrieved_ev[0:1]))

In [116]:
# from multiprocessing.pool import ThreadPool as Pool
# from yake import KeywordExtractor
# import tqdm.notebook as tqdm
# import time
# from summa import keywords
# from tqdm import tqdm
#
# ### PASSAGE RANKING; KEYWORD OVERLAP ###
# kw_extractor = KeywordExtractor(lan="en", n=3, top=5)
#
# # TODOs: For each ADU, Rank Merged Evidence using Keyword Overlap and Filter for Contrasting Stance
# # TODOs: Handel Multiple Keywords
#
# def overlap_score(evidence_kp, adu_kp):
#     score = 0
#     # TODOs: Robust 'None' handeling
#     if adu_kp == None:
#         return score
#     # Split Keyphrase into components, scoring partial units as overlap
#     else:
#         for i in evidence_kp:
#             for j in i.split():
#                 # Ensure string value, to enact .find
#                 if ", ".join([i for i in adu_kp]).find(j) != -1: score += 1
#
#                 else: continue
#
#     return score
#
# def calculate_overlap(merged_ev, adu_kp):
#
#     for ev_unit in sentences_segment(merged_ev):
#         toks = tokeniser(ev_unit)
#         kp_overlap = 0
#
#         if len(toks) <= 8: continue
#
#         #ev_unit_kp = [i for i in keywords.keywords(ev_unit).split("\n")]
#         ev_unit_kp = [i[0] for i in kw_extractor.extract_keywords(ev_unit)]
#
#         if ev_unit_kp:
#             kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
#
#         else: ev_unit_kp = None
#         yield ev_unit, ev_unit_kp, kp_overlap
#
# # pool = Pool(8)
# ### RANK PASSAGES ###
# def score_passages(ev_):
#     adu = ev_[0]["argument_discourse_unit"]
#     adu_stance = ev_[0]["adu_stance"]
#     merged_ev = ev_[0]["merged_evidence"]
#     adu_kp = ev_[0]["adu_keyphrases"]
#
#     ### CALCULATE OVERLAP ###
#     for ev_unit, ev_unit_kp, kp_overlap in calculate_overlap(merged_ev, adu_kp):
#         target = adu_kp[0]
#
#         compared_stace = compare_stance(ev_unit, target)
#         if compared_stace != adu_stance:
#             yield {
#                 "adu": adu,
#                 "adu_kp": adu_kp,
#                 "evidence_unit": ev_unit,
#                 "evidence_kps": ev_unit_kp,
#                 "overlap": kp_overlap,
#                 "evidence_stance": compare_stance(ev_unit, target),
#                 "adu_stance": adu_stance
#             }
#
#         else: continue
#
# ### SCORED EVIDENCE ###
# def score_evidence(retrieved_evidence):
#     for ev_ in retrieved_ev:
#         yield [i for i in score_passages(ev_)]
#
# ### RANKED EVIDENCE ###
# def rank_filter_counter_evidence(retireved_evidence, k=3):
#     with tqdm(total=(len(retrieved_ev))) as pbar:
#         for i in score_evidence(retrieved_ev):
#             yield sorted(i, key=lambda y: y["overlap"], reverse=True)[0:k]
#
#             pbar.update()
#
#
# ### SELECT TOP-K COUNTER-EVIDENCE ###
# tic = time.time()
# ranked_sorted_evidence = [i for i in rank_filter_counter_evidence(retrieved_ev)]
# ranked_sorted_evidence
# toc = time.time()
#
# print(toc - tic)
# # TIME 1:20M

100%|██████████| 100/100 [00:30<00:00,  3.23it/s]

30.97145128250122


In [250]:
# idx = 2
# for ln in retrieved_ev:
#     r = ln[0]
#     for _ in range(0, len(r["argument"])):
#         print(r["argument"][_]["sentence"])
#         print(r["argument"][_]["kp"])
#         print("")
#         print(r["retrieved"][_]["evidence"])
#         print(r["retrieved"][_]["kp"])

#"counter": {"counter": arg["counter"]["counter"], "counter_kp": arg["counter"]["counter_keyphrases"]}
# "argument_discourse_unit": adu,
# "query": query,
# "adu_keyphrases": [i for i in kp],
# "adu_stance": sentence_stance(adu, kp),
# "merged_evidence": ", ".join(ln for ln in evidence)
# "retrieved_documents_titles": titles,
# "retrieved_evidence": evidence,

Section I Why is Basic Income Increasingly Popular?
['basic income increasingly popular', 'basic income increasingly', 'section basic income increasingly']

Brazil. Minimum income has been increasingly accepted by the Brazilian government. In 2004, President Lula da Silva signed into law a bill to establish a universal basic income. Committee member Lady Rhys-Williams argued that the incomes for adults should be more like a basic income. She was also the first to develop the negative income tax model. Her son Brandon Rhys Williams proposed a basic income to a parliamentary committee in 1982, and soon after that in 1984, the Basic Income Research Group, now the Citizen's Basic Income Trust, began to conduct and disseminate research on basic income. Many technology experts and technology entrepreneurs have begun endorsing basic income in the 2000s and 2010s. These include Marshal Brain, Sam Altman, James Hughes, Facebook co-founder Chris Hughes, Elon Musk, and Mark Zuckerberg (in his 201

In [ ]:
# # TODOs: Speed-up, Parrelleise, Yield
# def overlap_score(evidence_kp, adu_kp):
#     score = 0

#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: score += 1

#             else: continue

#     return score

# ev_units = evidence
# adu_kp = extract_keyphrase(adu)

# adu_ev_overlap = []

# kp_1 = ['sex', 'relationship', 'opportunity']
# kp_2 = ['better sex']

# overlap_score(kp_2, kp_1)

# for ev_unit in evidence:
#     #print(ev_unit)
#     toks = tokeniser(ev_unit)

#     # Exprimental Value
#     if len(toks) <= 8:
#         continue

#     ev_unit_kp = extract_keyphrase(ev_unit)
#     kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)

#     adu_ev_overlap.append({
#         "adu": adu,
#         "adu_kp": adu_kp,
#         "ev_unit": ev_unit,
#         "ev_unit_kp": ev_unit_kp,
#         "kp_overlap": kp_overlap

#         })

# adu_ev_overlap

In [ ]:
# ### OVERLAP RANKED EVIDENCE ###

# adu_ev_overlap.sort(key=lambda y: y["kp_overlap"], reverse=True)
# adu_ev_overlap

# ### FILTER IRRELEVANT EVIDENCE ###
# overlapping = [i for i in adu_ev_overlap if i["kp_overlap"] !=0]

# len(adu_ev_overlap), len(overlapping)
# overlapping


In [15]:
# Stance Test
# adu = 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural.'
# ev_unit = 'In this essay, Mill argues the idea that the morality of an action can be judged by whether it is natural or unnatural.'
# target = 'natural things'
#
# stance = compare_stance(ev_unit, target)
# stance

'PRO'

In [ ]:
# ### ASSERT SAME STANCE ###
# from detection.stance_classifier import sentence_stance, compare_stance
#
# # TODOs: Ensure KPs Extracts are constrained to 1 unit
# opposing_stance = []
# for i in overlapping:
#     adu = i["adu"]
#     target = " ".join(i for i in i["adu_kp"])
#     ev_unit = i["ev_unit"]
#
#     ev_stance = compare_stance(ev_unit, ev_unit, target)
#     adu_stance = sentence_stance(adu, target)
#
#     if ev_stance != adu_stance:
#         opposing_stance.append((ev_unit, ev_stance, adu_stance))
#
#     else: continue
#
# opposing_stance

In [ ]:
### RANKING ###

# TODOs: Speed-up, Parrelleise, Yield
# ev_units = evidence
# adu_kp = extract_keyphrase(adu)

# adu_ev_overlap = []

# kp_1 = ['sex', 'relationship', 'opportunity'] 
# kp_2 = ['better sex']

# overlap_score(kp_2, kp_1)

# for ev_unit in evidence:
#     #print(ev_unit)
#     toks = tokeniser(ev_unit)

#     # Exprimental Value
#     if len(toks) <= 8:
#         continue
    
#     ev_unit_kp = extract_keyphrase(ev_unit)
#     kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
    
#     adu_ev_overlap.append({
#         "adu": adu, 
#         "adu_kp": adu_kp,
#         "ev_unit": ev_unit,
#         "ev_unit_kp": ev_unit_kp, 
#         "kp_overlap": kp_overlap
        
#         })
        
# adu_ev_overlap


#rank_passages(retrieved_ev)

In [ ]:
# import spacy
# from spacy.matcher import PhraseMatcher
# from fuzzywuzzy import fuzz, process

# # TODOs: Package as a Module
# # TODOs: Handle Negation (Polarity shifters)
# # TODOs: Review Unsuperived Approach; Consider adveanced patterns and common-sence knowledge

# nlp = spacy.load("en_core_web_sm")

# sentence = "I hate abortion rights. Abortions should be banned."
# sentence_2 = "I like abortion rights. I belive we should keep them."
# sentence_3 = "I hate tennis. People should play tennis more often"

# ### STANCE SCORING ###

# # TODOs: https://www.cs.uic.edu/~liub/FBS/opinion-mining-final-WSDM.pdf 
# # TODOs: Pattern based Negation
# # TODOs: Semantic Orientation of an opinion (Claim)
# # TODOs:Group synonyms of 'features', 'targets'

# phrase_matcher = PhraseMatcher(nlp.vocab)

# ### SENTIMENT LEXICONS ###
# pos = [w.replace("\n", "") for w in open("../../data/lexicon/positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("../../data/lexicon/negative_lex.txt")]
# polarity_shifters = [w.replace("\n", "") for w in open("../../data/lexicon/shifter_lexicon.txt")]

# ### STANCE: ASPECT-SEMANTIC ORIENTATION ###
# def extract_aspect(sentence, n_gram):
#     aspects = extract_keyphrase(str(sentence))[0]

#     return nlp(aspects)

# def index_aspect(phrase, aspect, sentence):    
#     patterns = [nlp(aspect)]
#     phrase_matcher.add(phrase, None, *patterns)

#     start = 0
#     stop = 0

#     matched_phrases = phrase_matcher(sentence)
#     for i in matched_phrases:
#         _, start, stop = i
        
#     return start, stop

# # TODOs: Implement Polarity Shifters, Simple
# # TODOs: Implement Polarity Shifters, Complex, Verb Patterns
# def stance_score(start, stop, sentence):
#     pos_score = 0.0
#     neg_score = 0.0

#     score = 0
#     for idx, tok in enumerate(sentence):
#         if idx == start or idx == stop:
#             continue

#         # TODOs: Implement Polarity Shift
#         # TODOs: Experiement with descriptive term + keyphrase aspects
#         # TODOs: ABSA https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis
#         # Use external libaray: Textblob
        
#         k = 8
#         # Negation Rules
#         shifted_tok = None
#         shifted_toks = []

#         if (tok.dep_ == "neg") or (tok.dep_ in polarity_shifters):
#             #Shift to Negative
#             if idx <= k:
#                 if idx < start: neg_score += 1/(start - idx)
#                 else: neg_score += 1/(idx - stop)**0.5

#             if shifted_tok != None and shifted_tok in neg:
#                 print(shifted_tok.text)
#                 # Shift to Positive
#                 if idx < start: pos_score += 1/(start - idx)
#                 elif idx > start: pos_score += 1/(idx - stop)**0.5
#                 else: continue

#         # Aspect Sentement Orientation
#         if tok.text in pos:
#             if tok in shifted_toks:
#                 continue
            
#             if idx < start: pos_score += 1/(start - idx)
#             else: pos_score += 1/(idx - stop)**0.5

#         if tok.text in neg:
#             if tok in shifted_toks:
#                 continue

#             if idx <= start: neg_score += 1/(start - idx)
#             else: neg_score += 1/(idx - stop)**0.5
    
#     score = pos_score - neg_score /(pos_score + neg_score + 1)

#     return score

# def overlap_score(evidence_kp, adu_kp):
#     score = 0
    
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: 
#                 score += 1
#                 token = j
            
#             else: continue
    
#     return score

# def get_overlapping_token(evidence_kp, adu_kp):
#     for i in evidence_kp:
#         overlap_tokens = []
#         for j in i.split():
#             if " ".join(adu_kp).find(j) != -1: 
#                 overlap_tokens.append(j) 
            
#         return " ".join(i for i in overlap_tokens)

# def sentence_stance(sentence, aspect):
#     sentence = nlp(sentence)

#     start, stop = index_aspect("aspects", aspect, sentence)
#     score = stance_score(start, stop, sentence)

#     # Add Neutral
#     #stance = {"claim": sentence, "stance": "PRO", "aspect": aspect} if score > 0 else {"claim": sentence, "stance": "CON", "aspect": aspect}
    
#     return "PRO" if score > 0 else "CON"

# def fuzzy_match(target, evidence_unit):

#     overlapping_aspect = process.extractOne(target, ev.split())[0]
#     score = overlapping_aspect[1]

#     overlapping_aspect = nlp(re.sub(r'[^\w]', ' ', overlapping_aspect))

#     return overlapping_aspect, score

# def compare_stance(ev_unit, evidence_aspect, adu_target):
#     # Note: Already identified mathcing or partially matching Aspects. 

#     # Get the overlapping evidence aspect-target.
#     overlapping_target, score = fuzzy_match(target=adu_aspect, evidence_unit=ev)
    
#     # Get position of the overlapping_target
#     start, stop = index_aspect("OVERLAP", nlp(overlapping_target), nlp(ev_unit))

#     # Assert Stance towards evidence aspect
#     score = stance_score(start, stop, nlp(ev_unit))
    
#     return "PRO" if score > 0 else "CON"

# ev = "These simple ideas and techniques could help both you and your lover enjoy sex. 1 / 10 Getty Images/Caiaimage Think beyond the thrust."
# ev_aspect = "sex", "relationship", "opportunity"

# adu = 'Hello! Let me preface by saying I dont believe there is a better sex.'
# adu_aspect = "better sex"

# print(sentence_stance("The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're not in a relationship.", adu_aspect))
# print(compare_stance(ev, ev_aspect, adu_aspect))


In [ ]:
# from spacy.matcher import DependencyMatcher, Matcher
# matcher = Matcher(vocab=nlp.vocab)
# matcher

# # Matching Rule: Pronouns with Verbs that follow them
# aspect = "better sex"
# patterns = [
#     [{"DEP": "neg"}, {"LOWER": aspect}],
#     [{"DEP": "neg"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"POS": "VERB"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"LOWER": aspect.lower()}]
# ]

# test = nlp("Hello! Let me preface by saying I dont believe there is a not better sex.")
# test_2 = nlp("These simple ideas and techniques could help both you and your lover enjoy better sex.")

# matcher.add("test", patterns=patterns)
# result = matcher(test_2, as_spans=True)

# result

# # for tok in test:
# #     print(tok.i, tok, tok.pos_, tok.dep_, tok.head.i, sep="\t")

In [ ]:
### TARGETED RETRIEVAL: ATTACKING PEMISES ###

# from BERT_adu_classifier import predict

# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)